In [21]:
import pandas as pd
import ast
qa_result_df = pd.read_csv("data/interim/qa_result_2023-07-19.csv")
# convert the 2nd column to dictionary by literal eval function and then convert to two columns
import re

import re

def correct_string_format(s):
    # Extract the title part from the string
    match = re.search(r'"title": (.+)}', s)
    if match is not None:  # check if match found
        start, end = match.span(1)
        # Remove existing double quotes
        removed_quotes_s = s[start:end].replace('"', '')
        # Put the value of "title" within quotes and replace the original in s
        corrected_s = s[:start] + '"' + removed_quotes_s + '"' + s[end:]
        return corrected_s
    else:
        return s  # return the original string if no match is found

qa_result_df['label_dict'] = qa_result_df.iloc[:,3].apply(correct_string_format)
qa_result_df["label_dict"] = qa_result_df['label_dict'].apply(lambda x: ast.literal_eval(x))
qa_result_df = qa_result_df.join(pd.json_normalize(qa_result_df["label_dict"]))
qa_result_df["summary"] = qa_result_df["summary"].apply(lambda x: ' '.join(x))

In [91]:
import pandas as pd
import ast
from geotext import GeoText
from geopy.geocoders import Nominatim
from tqdm import tqdm, notebook
def extract_location(text):
    # create a geocoder object
    geolocator = Nominatim(user_agent="my-app")
    location = geolocator.geocode(text)
    if location is not None:
        lat = location.latitude
        lon = location.longitude
        return lat, lon
    else:
        print("Got an error for ", text)    
        return None


qa_result_df = pd.read_csv("data/interim/qa_result_2023-07-19.csv").iloc[0:10,:]

qa_result_df["label_dict"] = qa_result_df.iloc[:,3].apply(lambda x: ast.literal_eval(x))
qa_result_df = qa_result_df.join(pd.json_normalize(qa_result_df["label_dict"]))
# set the first column as index
qa_result_df.set_index("file_name", inplace=True)
qa_result_df = qa_result_df["study_area"].explode().to_frame()
# create a string of "Country, City" for each paper
qa_result_df["study_area"] = qa_result_df["study_area"].apply(lambda x: ast.literal_eval(str(x)))
qa_result_df["study_area"] = qa_result_df["study_area"].apply(lambda x: x["Country"] + ", " + x["City"] if x.get("City") not in [None, "not mentioned", "not specified"] else x["Country"])
qa_result_df["study_area"] = qa_result_df["study_area"].apply(lambda x: extract_location(str(x)))
qa_result_df

,study_area
file_name,
10.1016_j.landurbplan.2022.104358.txt,"(31.2323437, 121.4691024)"
10.1016_j.jenvp.2023.102028.txt,"(39.084802, 117.1959904)"
10.1016_j.ufug.2022.127712.txt,"(60.1674881, 24.9427473)"
10.1016_j.ufug.2019.126477.txt,"(50.0874654, 14.4212535)"
10.1016_j.ufug.2020.126903.txt,"(31.2323437, 121.4691024)"
10.1016_j.buildenv.2021.108323.txt,"(29.8622194, 121.6203873)"
10.1016_j.ufug.2017.05.002.txt,"(53.9097123, -122.7623262)"
10.1016_j.cities.2022.103650.txt,"(52.2434979, 5.6343227)"
10.1016_j.jenvp.2019.03.005.txt,"(-34.135635300000004, -70.73998533381518)"


In [7]:
import pandas as pd
import ast
qa_result_df = pd.read_csv("data/interim/qa_result_2023-07-19.csv")
qa_result_df['label_dict'] = qa_result_df.iloc[:,8]
qa_result_df["label_dict"] = qa_result_df['label_dict'].apply(lambda x: ast.literal_eval(x))
qa_result_df = qa_result_df.join(pd.json_normalize(qa_result_df["label_dict"]))
qa_result_df.set_index("file_name", inplace=True)
qa_result_df = qa_result_df["research_types"].explode().to_frame()
qa_result_df["research_types"].unique()

array(['model development', 'index construction', 'exploratory analysis',
       'others: qualitative research', 'regression',
       'others: survey research', 'others: comparative analysis',
       'others: index construction', 'correlational design',
       'questionnaire design', 'pilot testing',
       'convenience and snow-ball sampling approach',
       'others: ordinal logit regression models',
       'others: model development', 'others: exploratory analysis',
       'others: quantitative analysis', 'others: pre-post design',
       'others: cluster analysis', 'others: mixed-methods',
       'others: visitor employed photography',
       'others: mediation analyses', 'others: Experimental Design',
       'others: Data Collection Instruments',
       'others: questionnaire design and survey context',
       'others: research variables and data collection procedure',
       'others: sampling size and sampling method',
       'others: survey procedure', 'others: data analysis',
 

In [5]:
import pandas as pd
import ast
qa_result_df = pd.read_csv("data/interim/qa_result_2023-07-19.csv")
qa_result_df['label_dict'] = qa_result_df.iloc[:,9]

# add {'cv': } around the string without it
def add_cv(x):
    if "cv" not in x:
        return "{'cv_models': " + x + "}"
    else:
        return x
qa_result_df["label_dict"] = qa_result_df["label_dict"].apply(lambda x: add_cv(x))
def add_brackets_to_string(s):
    # Locate the first occurrence of ':'
    index = s.find(':')
    if index == -1:
        # If there's no colon, return the original string (This should not happen in your example)
        return s
    
    # Add the opening bracket after the colon and space
    s = s[:index + 2] + "[" + s[index + 2:]
    
    # Add the closing bracket just before the last }
    # check if the last character is a }
    if s[-1] == '}':
        # If it is, add the bracket before it
        s = s[:-1] + "]" + s[-1:]
    else:
        s = None
    return s

def ast_literal_eval(x):
    try:
        return ast.literal_eval(x)
    except:
        x = add_brackets_to_string(x)
        if (x != "{'cv_models': [not applicable]}") & (x != None):
            return ast.literal_eval(x)
        else:
            return None

# check the length of the list and explode the list horizontally to columns
def explode_horizontally(x):
    if len(x) != 3:
        return None
    else:
        return x
# drop None
qa_result_df = qa_result_df[qa_result_df["label_dict"].notnull()]
qa_result_df["label_dict"] = qa_result_df['label_dict'].apply(lambda x: ast_literal_eval(x))
qa_result_df = qa_result_df.join(pd.json_normalize(qa_result_df["label_dict"]))
qa_result_df.set_index("file_name", inplace=True)
qa_result_df = qa_result_df["cv_models"].explode().to_frame().reset_index()
# Filter rows where the list has exactly 3 elements
qa_result_df = qa_result_df.dropna(subset=['cv_models'])
qa_result_df = qa_result_df[qa_result_df['cv_models'].apply(len) == 3]
# Explode the list column into multiple columns
# df_new = pd.DataFrame(qa_result_df['cv_models'].to_list(), columns=['col1_1', 'col1_2', 'col1_3'])
# Then you can convert the 'label_dict' column to a list and create your new DataFrame:
# qa_result_df['cv_models'].apply(lambda x: explode_horizontally(x))
qa_result_df = pd.DataFrame(qa_result_df['cv_models'].to_list(), index=qa_result_df["file_name"])
# Then you can convert the 'label_dict' column to a list and create your new DataFrame:
# qa_result_df['cv_models'].apply(lambda x: explode_horizontally(x))
# qa_result_df = pd.DataFrame(qa_result_df['cv_models'].to_list(), index=qa_result_df["file_name"])
qa_result_df

,0,1,2
file_name,,,
10.1016_j.landurbplan.2022.104358.txt,PSPNet,semantic segmentation,pre-trained with fine-tuning
10.1016_j.landurbplan.2022.104358.txt,Random Forest,feature extraction,trained from scratch by themselves
10.1016_j.landurbplan.2018.09.015.txt,SegNet,semantic segmentation,pre-trained without fine-tuning
10.1016_j.landurbplan.2022.104603.txt,Deeplab-v3+,semantic segmentation,pre-trained with fine-tuning
10.1016_j.landurbplan.2022.104603.txt,Tracktor++,object detection,pre-trained with fine-tuning
...,...,...,...
2733373.2806273.pdf,AlexNet,image classification,pre-trained with fine-tuning
2733373.2806273.pdf,RankingSVM,others,not applicable
2733373.2806273.pdf,CNN,image classification,trained from scratch by themselves


In [57]:
import pandas as pd

# Sample dataframe
data = {'col1': [[1, 2, 3], [4, 5], [6], [7, 8, 9, 10]]}
df = pd.DataFrame(data)

# Filter rows where the list has exactly 3 elements
df = df[df['col1'].apply(len) == 3]

# Explode the list column into multiple columns
df_new = pd.DataFrame(df['col1'].to_list(), columns=['col1_1', 'col1_2', 'col1_3'])

print(df_new)


   col1_1  col1_2  col1_3
0       1       2       3


In [9]:
import pandas as pd
import ast
def check_and_fix(x):
    # check if x starts and ends with {}
    if not x.startswith("{"):
        return "{" + x
    if not x.endswith("}"):
        return x + "}"
    return x
        
def ast_literal_eval(x):
    try:
        return ast.literal_eval(x)
    except:
        x = check_and_fix(x)
        return ast.literal_eval(x)    
qa_result_df = pd.read_csv("data/interim/qa_result_2023-07-19.csv")
qa_result_df['label_dict'] = qa_result_df.iloc[:,13]
qa_result_df["label_dict"] = qa_result_df['label_dict'].apply(lambda x: ast_literal_eval(x))
print(qa_result_df)

                                             file_name  \
0                10.1016_j.landurbplan.2022.104358.txt   
1                      10.1016_j.jenvp.2023.102028.txt   
2                       10.1016_j.ufug.2022.127712.txt   
3                       10.1016_j.ufug.2019.126477.txt   
4                       10.1016_j.ufug.2020.126903.txt   
..                                                 ...   
395                                2733373.2806273.pdf   
396                                 jurdp.19.00040.pdf   
397        30075-Article Text-108307-1-10-20220401.pdf   
398  Using Eye Tracking to Understand Human Respons...   
399                           978-3-031-22064-7_19.pdf   

    1. Answer DOI and title of the paper. Answer 'not mentioned' if you cannot find answers for DOI and title.\nExample Answer: \n------\nDOI: *XXX*\nTitle: *XXX*\n------  \
0    {\n  "doi": "not mentioned",\n  "title": "not ...                                                                           